In [1]:
import xarray as xr
import pandas as pd
from os.path import join
from sklearn.preprocessing import MinMaxScaler

In [2]:
ROOT_DIR = '/home/jovyan/crop-forecasting'
FOLDER = 'augment_50_5'

In [3]:
xdf_train = xr.open_dataset(join(ROOT_DIR, 'data', 'processed', FOLDER, 'train.nc'), engine='scipy')
df_train = pd.read_csv(join(ROOT_DIR, 'data', 'interim', 'train_enriched.csv'))
df_train.set_index(['Unnamed: 0'], inplace=True)
dictio = {'Unnamed: 0': xdf_train['ts_obs']}

scaler = MinMaxScaler()
scaler.fit(df_train[['Other Rice Yield (kg/ha)']])
df_train['Other Rice Yield (kg/ha)'] = scaler.transform(df_train[['Other Rice Yield (kg/ha)']])

xdf_train = xr.merge([xdf_train, df_train[['Other Rice Yield (kg/ha)']].to_xarray().sel(**dictio)], compat='override')
xdf_train = xdf_train.reset_coords('Unnamed: 0', drop=True)

In [4]:
xdf_test = xr.open_dataset(join(ROOT_DIR, 'data', 'processed', FOLDER, 'test.nc'), engine='scipy')
df_test = pd.read_csv(join(ROOT_DIR, 'data', 'interim', 'test_enriched.csv'))
df_test.set_index(['Unnamed: 0'], inplace=True)
dictio = {'Unnamed: 0': xdf_test['ts_obs']}

scaler.fit(df_test[['Other Rice Yield (kg/ha)']])
df_test['Other Rice Yield (kg/ha)'] = scaler.transform(df_test[['Other Rice Yield (kg/ha)']])

xdf_test = xr.merge([xdf_test, df_test[['Other Rice Yield (kg/ha)']].to_xarray().sel(**dictio)], compat='override')
xdf_test = xdf_test.reset_coords('Unnamed: 0', drop=True)

In [5]:
xdf_train.to_netcdf(join(ROOT_DIR, 'data', 'processed', FOLDER, 'train_enriched.nc'), engine='scipy')
xdf_test.to_netcdf(join(ROOT_DIR, 'data', 'processed', FOLDER, 'test_enriched.nc'), engine='scipy')